In [4]:
pip install rdflib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import os
import requests
import zipfile
import subprocess
import shutil
from rdflib import Graph

# Tạo file univ-bench.owl
def create_univ_bench_ontology():
    ontology_path = "LUBM_Data/uba/univ-bench.owl"
    if not os.path.exists(ontology_path):
        print("Đang tạo tệp univ-bench.owl...")
        ontology_content = """<?xml version="1.0"?>
<rdf:RDF xmlns="http://www.lehigh.edu/~zhp2/2004/0401/univ-bench.owl#"
         xml:base="http://www.lehigh.edu/~zhp2/2004/0401/univ-bench.owl"
         xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
         xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#"
         xmlns:owl="http://www.w3.org/2002/07/owl#"
         xmlns:xsd="http://www.w3.org/2001/XMLSchema#">
  <owl:Ontology rdf:about=""/>
  <rdf:Description rdf:about="http://www.lehigh.edu/~zhp2/2004/0401/univ-bench.owl#University">
    <rdf:type rdf:resource="http://www.w3.org/2002/07/owl#Class"/>
  </rdf:Description>
  <!-- Bạn có thể thêm nhiều định nghĩa và mô tả hơn nếu cần -->
</rdf:RDF>"""
        
        os.makedirs(os.path.dirname(ontology_path), exist_ok=True)
        with open(ontology_path, "w", encoding="utf-8") as f:
            f.write(ontology_content)
        print("Tạo tệp univ-bench.owl thành công.")
    else:
        print("Tệp univ-bench.owl đã tồn tại.")


# URL để tải xuống công cụ LUBM UBA
url = "http://swat.cse.lehigh.edu/projects/lubm/uba1.7.zip"
zip_file_path = "LUBM_Data/uba1.7.zip"
extract_folder = "LUBM_Data/uba"

# Tải về công cụ LUBM UBA
def download_lubm_uba():


    # Tạo thư mục cho dự án
    os.makedirs("LUBM_Data", exist_ok=True)

    # Tải về tệp zip
    print("Đang tải công cụ LUBM UBA...")
    response = requests.get(url)
    if response.status_code == 200:
        with open(zip_file_path, "wb") as f:
            f.write(response.content)
        print("Tải về thành công.")
    else:
        print("Có lỗi xảy ra khi tải tệp.")
        return False

    # Giải nén tệp zip
    print("Đang giải nén tệp...")
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_folder)
    print("Giải nén thành công.")

    return True

# Tạo JAR từ các file class đã biên dịch với thông tin MANIFEST
import os
import subprocess

# Tạo JAR từ các file class đã biên dịch với thông tin MANIFEST
def create_jar():
    class_path = os.path.join(extract_folder, "classes")
    uba_jar_path = os.path.join(class_path, "uba.jar")
    manifest_path = os.path.join(class_path, "MANIFEST.MF")
    
    # Tạo tệp MANIFEST
    with open(manifest_path, "w") as manifest_file:
        manifest_file.write("Manifest-Version: 1.0\n")
        manifest_file.write("Main-Class: edu.lehigh.swat.bench.uba.Generator\n")
    
    if not os.path.exists(uba_jar_path):
        print("Đang tạo tệp uba.jar từ các tệp class với thông tin MANIFEST...")
        command = ["jar", "cfm", uba_jar_path, manifest_path, "-C", class_path, "."]
        try:
            subprocess.run(command, check=True)
            print("Tạo uba.jar thành công.")
        except subprocess.CalledProcessError as e:
            print(f"Lỗi khi tạo uba.jar: {e}")
            return False
    return True

# Tạo dữ liệu OWL sử dụng LUBM UBA
def generate_lubm_data(num_universities=5):
    # Đường dẫn tới tệp uba.jar
    uba_jar_path = os.path.abspath(r"LUBM_Data/uba/classes/uba.jar")
    # Đường dẫn tới tệp ontology, thay đổi để có định dạng phù hợp
    ontology_path = os.path.abspath(r"LUBM_Data/uba/univ-bench.owl").replace("\\", "/")

    # Kiểm tra xem tệp JAR có tồn tại tại đường dẫn này hay không
    if not os.path.exists(uba_jar_path):
        print(f"Lỗi: Không tìm thấy tệp JAR tại đường dẫn {uba_jar_path}")
        return

    # Kiểm tra xem tệp ontology có tồn tại hay không
    if not os.path.exists(ontology_path):
        print(f"Lỗi: Không tìm thấy tệp ontology tại đường dẫn {ontology_path}")
        return

    # Câu lệnh để chạy uba.jar với số lượng trường đại học mong muốn và chỉ định ontology
    command = f'java -jar "{uba_jar_path}" -univ {num_universities} -onto file:"{ontology_path}"'
    try:
        print(f"Đang tạo dữ liệu OWL cho {num_universities} trường đại học...")
        result = subprocess.run(command, check=True, shell=True)
        print("Dữ liệu OWL đã được tạo thành công.")
    except subprocess.CalledProcessError as e:
        print(f"Lỗi khi tạo dữ liệu LUBM: {e}")

# Di chuyển các tệp OWL đến thư mục đầu ra
def move_owl_files_to_output():
    current_directory = os.getcwd()
    output_directory = os.path.abspath(r"LUBM_Data/data")

    # Tạo thư mục đầu ra nếu nó chưa tồn tại
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    # Tìm và di chuyển các tệp OWL
    for file_name in os.listdir(current_directory):
        if file_name.endswith(".owl"):
            source_path = os.path.join(current_directory, file_name)
            destination_path = os.path.join(output_directory, file_name)
            shutil.move(source_path, destination_path)
            print(f"Đã di chuyển {file_name} đến {output_directory}")

# Chuyển đổi OWL sang RDF
def convert_owl_to_rdf(input_directory, output_directory):
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    for file_name in os.listdir(input_directory):
        if file_name.endswith(".owl"):
            input_path = os.path.join(input_directory, file_name)
            output_path = os.path.join(output_directory, file_name.replace(".owl", ".rdf"))
            g = Graph()
            # Sử dụng URI hợp lệ cho ontology
            g.parse(input_path, format='xml')
            g.serialize(destination=output_path, format='xml')
            print(f"Đã chuyển đổi {file_name} thành {output_path}")

# Thực hiện tải và tạo dữ liệu LUBM
if download_lubm_uba():
    create_univ_bench_ontology()
    if create_jar():
        generate_lubm_data(num_universities=5)
        # Di chuyển các tệp OWL đến thư mục đầu ra
        move_owl_files_to_output()
        # Chuyển đổi OWL sang RDF
        convert_owl_to_rdf("LUBM_Data/data", "LUBM_Data/data")


Đang tải công cụ LUBM UBA...
Tải về thành công.
Đang giải nén tệp...
Giải nén thành công.
Tệp univ-bench.owl đã tồn tại.
Đang tạo dữ liệu OWL cho 5 trường đại học...
Dữ liệu OWL đã được tạo thành công.
Đã di chuyển University0_0.owl đến c:\Users\DELL\Desktop\Linh\CaoHoc\T6.GiaiThuat\rtree_kdtree_for_rdf\data\LUBM_Data\data
Đã di chuyển University0_1.owl đến c:\Users\DELL\Desktop\Linh\CaoHoc\T6.GiaiThuat\rtree_kdtree_for_rdf\data\LUBM_Data\data
Đã di chuyển University0_10.owl đến c:\Users\DELL\Desktop\Linh\CaoHoc\T6.GiaiThuat\rtree_kdtree_for_rdf\data\LUBM_Data\data
Đã di chuyển University0_11.owl đến c:\Users\DELL\Desktop\Linh\CaoHoc\T6.GiaiThuat\rtree_kdtree_for_rdf\data\LUBM_Data\data
Đã di chuyển University0_12.owl đến c:\Users\DELL\Desktop\Linh\CaoHoc\T6.GiaiThuat\rtree_kdtree_for_rdf\data\LUBM_Data\data
Đã di chuyển University0_13.owl đến c:\Users\DELL\Desktop\Linh\CaoHoc\T6.GiaiThuat\rtree_kdtree_for_rdf\data\LUBM_Data\data
Đã di chuyển University0_14.owl đến c:\Users\DELL\Deskto

In [6]:
import os
import re

def correct_ontology_paths(input_directory):

    base_path = os.getcwd()  # Đường dẫn thư mục hiện tại
    base_path_uri = "file:///" + base_path.replace("\\", "/") + '/LUBM_Data/data/' + base_path.replace("\\", "/") + '/LUBM_Data/uba/univ-bench.owl#'  # Chuyển đổi thành dạng URI
    base_path_uri_rep = "file:///" + base_path.replace("\\", "/") + '/LUBM_Data/uba/univ-bench.owl#' 

    for file_name in os.listdir(input_directory):
        if file_name.endswith(".rdf"):
            file_path = os.path.join(input_directory, file_name)
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()

            # Thay thế đường dẫn bị lặp bằng URI chuẩn
            corrected_content = content.replace(base_path_uri, base_path_uri_rep)

            # Ghi lại nội dung đã sửa vào tệp gốc
            with open(file_path, 'w', encoding='utf-8') as file:
                file.write(corrected_content)

            print(f"Đã sửa đường dẫn trong tệp {file_name}")

# Thực hiện sửa các tệp RDF trong thư mục đầu ra
correct_ontology_paths("LUBM_Data/data")


Đã sửa đường dẫn trong tệp University0_0.rdf
Đã sửa đường dẫn trong tệp University0_1.rdf
Đã sửa đường dẫn trong tệp University0_10.rdf
Đã sửa đường dẫn trong tệp University0_11.rdf
Đã sửa đường dẫn trong tệp University0_12.rdf
Đã sửa đường dẫn trong tệp University0_13.rdf
Đã sửa đường dẫn trong tệp University0_14.rdf
Đã sửa đường dẫn trong tệp University0_2.rdf
Đã sửa đường dẫn trong tệp University0_3.rdf
Đã sửa đường dẫn trong tệp University0_4.rdf
Đã sửa đường dẫn trong tệp University0_5.rdf
Đã sửa đường dẫn trong tệp University0_6.rdf
Đã sửa đường dẫn trong tệp University0_7.rdf
Đã sửa đường dẫn trong tệp University0_8.rdf
Đã sửa đường dẫn trong tệp University0_9.rdf
Đã sửa đường dẫn trong tệp University1_0.rdf
Đã sửa đường dẫn trong tệp University1_1.rdf
Đã sửa đường dẫn trong tệp University1_10.rdf
Đã sửa đường dẫn trong tệp University1_11.rdf
Đã sửa đường dẫn trong tệp University1_12.rdf
Đã sửa đường dẫn trong tệp University1_13.rdf
Đã sửa đường dẫn trong tệp University1_14.rdf


In [7]:
import os
import re

# Hàm chuẩn hóa tệp RDF

def normalize_rdf_files(input_directory):
    for file_name in os.listdir(input_directory):
        if file_name.endswith(".rdf"):
            file_path = os.path.join(input_directory, file_name)
            with open(file_path, 'r', encoding='utf-8') as file:
                lines = file.readlines()

            # Kiểm tra và xóa dòng trống ở đầu nếu có khai báo XML ở dòng sau
            while len(lines) > 1 and lines[0].strip() == "" and lines[1].startswith("<?xml"):
                del lines[0]

            # Ghi lại nội dung đã chuẩn hóa vào tệp gốc
            with open(file_path, 'w', encoding='utf-8') as file:
                file.writelines(lines)

            print(f"Đã chuẩn hóa tệp {file_name}")

# Chuẩn hóa các tệp RDF
normalize_rdf_files("LUBM_Data/data")


Đã chuẩn hóa tệp University0_0.rdf
Đã chuẩn hóa tệp University0_1.rdf
Đã chuẩn hóa tệp University0_10.rdf
Đã chuẩn hóa tệp University0_11.rdf
Đã chuẩn hóa tệp University0_12.rdf
Đã chuẩn hóa tệp University0_13.rdf
Đã chuẩn hóa tệp University0_14.rdf
Đã chuẩn hóa tệp University0_2.rdf
Đã chuẩn hóa tệp University0_3.rdf
Đã chuẩn hóa tệp University0_4.rdf
Đã chuẩn hóa tệp University0_5.rdf
Đã chuẩn hóa tệp University0_6.rdf
Đã chuẩn hóa tệp University0_7.rdf
Đã chuẩn hóa tệp University0_8.rdf
Đã chuẩn hóa tệp University0_9.rdf
Đã chuẩn hóa tệp University1_0.rdf
Đã chuẩn hóa tệp University1_1.rdf
Đã chuẩn hóa tệp University1_10.rdf
Đã chuẩn hóa tệp University1_11.rdf
Đã chuẩn hóa tệp University1_12.rdf
Đã chuẩn hóa tệp University1_13.rdf
Đã chuẩn hóa tệp University1_14.rdf
Đã chuẩn hóa tệp University1_15.rdf
Đã chuẩn hóa tệp University1_16.rdf
Đã chuẩn hóa tệp University1_17.rdf
Đã chuẩn hóa tệp University1_18.rdf
Đã chuẩn hóa tệp University1_2.rdf
Đã chuẩn hóa tệp University1_3.rdf
Đã chu

In [8]:
import os

# Dữ liệu RDF cần ghi vào file
rdf_data = """<?xml version="1.0" encoding="utf-8"?>
<rdf:RDF
   xmlns:ns1="#"
   xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
>
  <!-- Smith Relationships -->
  <rdf:Description rdf:about="Smith">
    <ns1:knows rdf:resource="Lucy"/>
    <ns1:knows rdf:resource="Mary"/>
    <ns1:knows rdf:resource="Julia"/>
  </rdf:Description>

  <!-- Brown Relationships -->
  <rdf:Description rdf:about="Brown">
    <ns1:knows rdf:resource="Holis"/>
    <ns1:knows rdf:resource="Tina"/>
  </rdf:Description>

  <!-- Maya Relationships -->
  <rdf:Description rdf:about="Maya">
    <ns1:knows rdf:resource="Lucy"/>
    <ns1:knows rdf:resource="Mary"/>
    <ns1:knows rdf:resource="Tina"/>
  </rdf:Description>

  <!-- Lucy Projects -->
  <rdf:Description rdf:about="Lucy">
    <ns1:project rdf:resource="LinkedData"/>
    <ns1:project rdf:resource="SemanticWeb"/>
  </rdf:Description>

  <!-- Julia Projects -->
  <rdf:Description rdf:about="Julia">
    <ns1:project rdf:resource="Ontology"/>
    <ns1:project rdf:resource="LinkedData"/>
    <ns1:project rdf:resource="SemanticWeb"/>
  </rdf:Description>

  <!-- Holis Projects -->
  <rdf:Description rdf:about="Holis">
    <ns1:project rdf:resource="Ontology"/>
    <ns1:project rdf:resource="DataMining"/>
  </rdf:Description>

  <!-- Tina Projects -->
  <rdf:Description rdf:about="Tina">
    <ns1:project rdf:resource="LinkedData"/>
    <ns1:project rdf:resource="BigData"/>
  </rdf:Description>

</rdf:RDF>"""

# Tạo thư mục và ghi dữ liệu vào file
os.makedirs("LUBM_Data/data_demo", exist_ok=True)  # Tạo thư mục data_demo nếu chưa tồn tại

# Đường dẫn của file RDF
file_path = os.path.join("LUBM_Data/data_demo", "Data_demo_1.rdf")

# Ghi dữ liệu RDF vào file
with open(file_path, "w", encoding="utf-8") as rdf_file:
    rdf_file.write(rdf_data)

print(f"File '{file_path}' đã được tạo thành công với dữ liệu RDF.")


File 'LUBM_Data/data_demo\Data_demo_1.rdf' đã được tạo thành công với dữ liệu RDF.


In [4]:
import os

# Dữ liệu RDF cần ghi vào file
rdf_data_test = """<?xml version="1.0" encoding="utf-8"?>
<rdf:RDF
   xmlns:ns1="#"
   xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
>
  <rdf:Description rdf:about="Student457">
    <ns1:memberOf rdf:resource="University0"/>
    <ns1:emailAddress>Student457@University0.edu</ns1:emailAddress>
    <ns1:takesCourse rdf:resource="Course37"/>
    <ns1:takesCourse rdf:resource="Course0"/>
  </rdf:Description>
  <rdf:Description rdf:about="FullProfessor6">
    <ns1:teacherOf rdf:resource="Course37"/>
    <ns1:DegreeFrom rdf:resource="University297"/>
    <ns1:worksFor rdf:resource="University0"/>
    <ns1:researchInterest>Research10</ns1:researchInterest>
  </rdf:Description>
  <rdf:Description rdf:about="Student23">
    <ns1:memberOf rdf:resource="University0"/>
    <ns1:emailAddress>Student23@University0.edu</ns1:emailAddress>
    <ns1:takesCourse rdf:resource="Course37"/>
  </rdf:Description>
</rdf:RDF>"""

# Tạo thư mục và ghi dữ liệu vào file
os.makedirs("LUBM_Data/data_test", exist_ok=True)  # Tạo thư mục data_test nếu chưa tồn tại

# Đường dẫn của file RDF
file_path_test = os.path.join("LUBM_Data/data_test", "University0_0.rdf")

# Ghi dữ liệu RDF vào file
with open(file_path_test, "w", encoding="utf-8") as rdf_file:
    rdf_file.write(rdf_data_test)

print(f"File '{file_path_test}' đã được tạo thành công với dữ liệu RDF.")


File 'LUBM_Data/data_test\University0_0.rdf' đã được tạo thành công với dữ liệu RDF.


In [1]:
import os

# Dữ liệu RDF cần ghi vào file
rdf_data_test = """<?xml version="1.0" encoding="utf-8"?>
<rdf:RDF
   xmlns:ns1="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#"
   xmlns:owl="http://www.w3.org/2002/07/owl#"
   xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/UndergraduateStudent358">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#UndergraduateStudent"/>
    <ns1:name>UndergraduateStudent358</ns1:name>
    <ns1:memberOf rdf:resource="http://www.Department1.University0.edu"/>
    <ns1:emailAddress>UndergraduateStudent358@Department1.University0.edu</ns1:emailAddress>
    <ns1:telephone>xxx-xxx-xxxx</ns1:telephone>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course31"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course47"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course41"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course22"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/AssociateProfessor8/Publication14">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#Publication"/>
    <ns1:name>Publication14</ns1:name>
    <ns1:publicationAuthor rdf:resource="http://www.Department1.University0.edu/AssociateProfessor8"/>
    <ns1:publicationAuthor rdf:resource="http://www.Department1.University0.edu/GraduateStudent55"/>
    <ns1:publicationAuthor rdf:resource="http://www.Department1.University0.edu/GraduateStudent107"/>
    <ns1:publicationAuthor rdf:resource="http://www.Department1.University0.edu/GraduateStudent109"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/UndergraduateStudent274">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#UndergraduateStudent"/>
    <ns1:name>UndergraduateStudent274</ns1:name>
    <ns1:memberOf rdf:resource="http://www.Department1.University0.edu"/>
    <ns1:emailAddress>UndergraduateStudent274@Department1.University0.edu</ns1:emailAddress>
    <ns1:telephone>xxx-xxx-xxxx</ns1:telephone>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course17"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course15"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/Lecturer1">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#Lecturer"/>
    <ns1:name>Lecturer1</ns1:name>
    <ns1:teacherOf rdf:resource="http://www.Department1.University0.edu/Course38"/>
    <ns1:teacherOf rdf:resource="http://www.Department1.University0.edu/Course39"/>
    <ns1:teacherOf rdf:resource="http://www.Department1.University0.edu/GraduateCourse39"/>
    <ns1:undergraduateDegreeFrom rdf:resource="http://www.University674.edu"/>
    <ns1:mastersDegreeFrom rdf:resource="http://www.University560.edu"/>
    <ns1:doctoralDegreeFrom rdf:resource="http://www.University132.edu"/>
    <ns1:worksFor rdf:resource="http://www.Department1.University0.edu"/>
    <ns1:emailAddress>Lecturer1@Department1.University0.edu</ns1:emailAddress>
    <ns1:telephone>xxx-xxx-xxxx</ns1:telephone>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/UndergraduateStudent368">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#UndergraduateStudent"/>
    <ns1:name>UndergraduateStudent368</ns1:name>
    <ns1:memberOf rdf:resource="http://www.Department1.University0.edu"/>
    <ns1:emailAddress>UndergraduateStudent368@Department1.University0.edu</ns1:emailAddress>
    <ns1:telephone>xxx-xxx-xxxx</ns1:telephone>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course20"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course46"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course40"/>
    <ns1:advisor rdf:resource="http://www.Department1.University0.edu/FullProfessor9"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.University100.edu">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#University"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/GraduateStudent84">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#GraduateStudent"/>
    <ns1:name>GraduateStudent84</ns1:name>
    <ns1:memberOf rdf:resource="http://www.Department1.University0.edu"/>
    <ns1:emailAddress>GraduateStudent84@Department1.University0.edu</ns1:emailAddress>
    <ns1:telephone>xxx-xxx-xxxx</ns1:telephone>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/GraduateCourse34"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/GraduateCourse13"/>
    <ns1:undergraduateDegreeFrom rdf:resource="http://www.University1.edu"/>
    <ns1:advisor rdf:resource="http://www.Department1.University0.edu/FullProfessor8"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/GraduateCourse0">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#GraduateCourse"/>
    <ns1:name>GraduateCourse0</ns1:name>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/UndergraduateStudent384">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#UndergraduateStudent"/>
    <ns1:name>UndergraduateStudent384</ns1:name>
    <ns1:memberOf rdf:resource="http://www.Department1.University0.edu"/>
    <ns1:emailAddress>UndergraduateStudent384@Department1.University0.edu</ns1:emailAddress>
    <ns1:telephone>xxx-xxx-xxxx</ns1:telephone>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course43"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course15"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course32"/>
    <ns1:advisor rdf:resource="http://www.Department1.University0.edu/FullProfessor1"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/UndergraduateStudent122">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#UndergraduateStudent"/>
    <ns1:name>UndergraduateStudent122</ns1:name>
    <ns1:memberOf rdf:resource="http://www.Department1.University0.edu"/>
    <ns1:emailAddress>UndergraduateStudent122@Department1.University0.edu</ns1:emailAddress>
    <ns1:telephone>xxx-xxx-xxxx</ns1:telephone>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course40"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course31"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course13"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/AssociateProfessor8/Publication12">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#Publication"/>
    <ns1:name>Publication12</ns1:name>
    <ns1:publicationAuthor rdf:resource="http://www.Department1.University0.edu/AssociateProfessor8"/>
    <ns1:publicationAuthor rdf:resource="http://www.Department1.University0.edu/GraduateStudent33"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/Course35">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#Course"/>
    <ns1:name>Course35</ns1:name>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/UndergraduateStudent133">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#UndergraduateStudent"/>
    <ns1:name>UndergraduateStudent133</ns1:name>
    <ns1:memberOf rdf:resource="http://www.Department1.University0.edu"/>
    <ns1:emailAddress>UndergraduateStudent133@Department1.University0.edu</ns1:emailAddress>
    <ns1:telephone>xxx-xxx-xxxx</ns1:telephone>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course33"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course11"/>
    <ns1:advisor rdf:resource="http://www.Department1.University0.edu/FullProfessor4"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/GraduateStudent20">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#GraduateStudent"/>
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#TeachingAssistant"/>
    <ns1:name>GraduateStudent20</ns1:name>
    <ns1:memberOf rdf:resource="http://www.Department1.University0.edu"/>
    <ns1:emailAddress>GraduateStudent20@Department1.University0.edu</ns1:emailAddress>
    <ns1:telephone>xxx-xxx-xxxx</ns1:telephone>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/GraduateCourse41"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/GraduateCourse5"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/GraduateCourse14"/>
    <ns1:undergraduateDegreeFrom rdf:resource="http://www.University165.edu"/>
    <ns1:advisor rdf:resource="http://www.Department1.University0.edu/AssistantProfessor1"/>
    <ns1:teachingAssistantOf rdf:resource="http://www.Department1.University0.edu/Course26"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/FullProfessor5/Publication18">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#Publication"/>
    <ns1:name>Publication18</ns1:name>
    <ns1:publicationAuthor rdf:resource="http://www.Department1.University0.edu/FullProfessor5"/>
    <ns1:publicationAuthor rdf:resource="http://www.Department1.University0.edu/GraduateStudent96"/>
    <ns1:publicationAuthor rdf:resource="http://www.Department1.University0.edu/GraduateStudent109"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/AssociateProfessor3/Publication3">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#Publication"/>
    <ns1:name>Publication3</ns1:name>
    <ns1:publicationAuthor rdf:resource="http://www.Department1.University0.edu/AssociateProfessor3"/>
    <ns1:publicationAuthor rdf:resource="http://www.Department1.University0.edu/GraduateStudent13"/>
    <ns1:publicationAuthor rdf:resource="http://www.Department1.University0.edu/GraduateStudent67"/>
    <ns1:publicationAuthor rdf:resource="http://www.Department1.University0.edu/GraduateStudent107"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/GraduateStudent23">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#GraduateStudent"/>
    <ns1:name>GraduateStudent23</ns1:name>
    <ns1:memberOf rdf:resource="http://www.Department1.University0.edu"/>
    <ns1:emailAddress>GraduateStudent23@Department1.University0.edu</ns1:emailAddress>
    <ns1:telephone>xxx-xxx-xxxx</ns1:telephone>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/GraduateCourse42"/>
    <ns1:undergraduateDegreeFrom rdf:resource="http://www.University760.edu"/>
    <ns1:advisor rdf:resource="http://www.Department1.University0.edu/AssistantProfessor2"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/UndergraduateStudent53">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#UndergraduateStudent"/>
    <ns1:name>UndergraduateStudent53</ns1:name>
    <ns1:memberOf rdf:resource="http://www.Department1.University0.edu"/>
    <ns1:emailAddress>UndergraduateStudent53@Department1.University0.edu</ns1:emailAddress>
    <ns1:telephone>xxx-xxx-xxxx</ns1:telephone>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course25"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course39"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/UndergraduateStudent171">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#UndergraduateStudent"/>
    <ns1:name>UndergraduateStudent171</ns1:name>
    <ns1:memberOf rdf:resource="http://www.Department1.University0.edu"/>
    <ns1:emailAddress>UndergraduateStudent171@Department1.University0.edu</ns1:emailAddress>
    <ns1:telephone>xxx-xxx-xxxx</ns1:telephone>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course35"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course36"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course6"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/UndergraduateStudent173">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#UndergraduateStudent"/>
    <ns1:name>UndergraduateStudent173</ns1:name>
    <ns1:memberOf rdf:resource="http://www.Department1.University0.edu"/>
    <ns1:emailAddress>UndergraduateStudent173@Department1.University0.edu</ns1:emailAddress>
    <ns1:telephone>xxx-xxx-xxxx</ns1:telephone>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course29"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course45"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course3"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/FullProfessor5/Publication5">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#Publication"/>
    <ns1:name>Publication5</ns1:name>
    <ns1:publicationAuthor rdf:resource="http://www.Department1.University0.edu/FullProfessor5"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/UndergraduateStudent357">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#UndergraduateStudent"/>
    <ns1:name>UndergraduateStudent357</ns1:name>
    <ns1:memberOf rdf:resource="http://www.Department1.University0.edu"/>
    <ns1:emailAddress>UndergraduateStudent357@Department1.University0.edu</ns1:emailAddress>
    <ns1:telephone>xxx-xxx-xxxx</ns1:telephone>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course44"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course30"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course31"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/UndergraduateStudent281">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#UndergraduateStudent"/>
    <ns1:name>UndergraduateStudent281</ns1:name>
    <ns1:memberOf rdf:resource="http://www.Department1.University0.edu"/>
    <ns1:emailAddress>UndergraduateStudent281@Department1.University0.edu</ns1:emailAddress>
    <ns1:telephone>xxx-xxx-xxxx</ns1:telephone>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course43"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course8"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course9"/>
    <ns1:advisor rdf:resource="http://www.Department1.University0.edu/FullProfessor1"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/UndergraduateStudent311">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#UndergraduateStudent"/>
    <ns1:name>UndergraduateStudent311</ns1:name>
    <ns1:memberOf rdf:resource="http://www.Department1.University0.edu"/>
    <ns1:emailAddress>UndergraduateStudent311@Department1.University0.edu</ns1:emailAddress>
    <ns1:telephone>xxx-xxx-xxxx</ns1:telephone>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course44"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course28"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/UndergraduateStudent7">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#UndergraduateStudent"/>
    <ns1:name>UndergraduateStudent7</ns1:name>
    <ns1:memberOf rdf:resource="http://www.Department1.University0.edu"/>
    <ns1:emailAddress>UndergraduateStudent7@Department1.University0.edu</ns1:emailAddress>
    <ns1:telephone>xxx-xxx-xxxx</ns1:telephone>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course6"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course39"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course23"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course41"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/UndergraduateStudent52">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#UndergraduateStudent"/>
    <ns1:name>UndergraduateStudent52</ns1:name>
    <ns1:memberOf rdf:resource="http://www.Department1.University0.edu"/>
    <ns1:emailAddress>UndergraduateStudent52@Department1.University0.edu</ns1:emailAddress>
    <ns1:telephone>xxx-xxx-xxxx</ns1:telephone>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course3"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course42"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course32"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course43"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/UndergraduateStudent141">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#UndergraduateStudent"/>
    <ns1:name>UndergraduateStudent141</ns1:name>
    <ns1:memberOf rdf:resource="http://www.Department1.University0.edu"/>
    <ns1:emailAddress>UndergraduateStudent141@Department1.University0.edu</ns1:emailAddress>
    <ns1:telephone>xxx-xxx-xxxx</ns1:telephone>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course10"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course2"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/UndergraduateStudent303">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#UndergraduateStudent"/>
    <ns1:name>UndergraduateStudent303</ns1:name>
    <ns1:memberOf rdf:resource="http://www.Department1.University0.edu"/>
    <ns1:emailAddress>UndergraduateStudent303@Department1.University0.edu</ns1:emailAddress>
    <ns1:telephone>xxx-xxx-xxxx</ns1:telephone>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course16"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course24"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course13"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course37"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/UndergraduateStudent125">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#UndergraduateStudent"/>
    <ns1:name>UndergraduateStudent125</ns1:name>
    <ns1:memberOf rdf:resource="http://www.Department1.University0.edu"/>
    <ns1:emailAddress>UndergraduateStudent125@Department1.University0.edu</ns1:emailAddress>
    <ns1:telephone>xxx-xxx-xxxx</ns1:telephone>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course5"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course24"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course17"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/UndergraduateStudent115">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#UndergraduateStudent"/>
    <ns1:name>UndergraduateStudent115</ns1:name>
    <ns1:memberOf rdf:resource="http://www.Department1.University0.edu"/>
    <ns1:emailAddress>UndergraduateStudent115@Department1.University0.edu</ns1:emailAddress>
    <ns1:telephone>xxx-xxx-xxxx</ns1:telephone>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course0"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course21"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course23"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course45"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/AssociateProfessor2/Publication8">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#Publication"/>
    <ns1:name>Publication8</ns1:name>
    <ns1:publicationAuthor rdf:resource="http://www.Department1.University0.edu/AssociateProfessor2"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/UndergraduateStudent283">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#UndergraduateStudent"/>
    <ns1:name>UndergraduateStudent283</ns1:name>
    <ns1:memberOf rdf:resource="http://www.Department1.University0.edu"/>
    <ns1:emailAddress>UndergraduateStudent283@Department1.University0.edu</ns1:emailAddress>
    <ns1:telephone>xxx-xxx-xxxx</ns1:telephone>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course40"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course12"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course11"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course8"/>
    <ns1:advisor rdf:resource="http://www.Department1.University0.edu/FullProfessor1"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/UndergraduateStudent32">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#UndergraduateStudent"/>
    <ns1:name>UndergraduateStudent32</ns1:name>
    <ns1:memberOf rdf:resource="http://www.Department1.University0.edu"/>
    <ns1:emailAddress>UndergraduateStudent32@Department1.University0.edu</ns1:emailAddress>
    <ns1:telephone>xxx-xxx-xxxx</ns1:telephone>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course0"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course8"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course45"/>
    <ns1:advisor rdf:resource="http://www.Department1.University0.edu/AssistantProfessor5"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/UndergraduateStudent178">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#UndergraduateStudent"/>
    <ns1:name>UndergraduateStudent178</ns1:name>
    <ns1:memberOf rdf:resource="http://www.Department1.University0.edu"/>
    <ns1:emailAddress>UndergraduateStudent178@Department1.University0.edu</ns1:emailAddress>
    <ns1:telephone>xxx-xxx-xxxx</ns1:telephone>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course16"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course47"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course10"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/UndergraduateStudent290">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#UndergraduateStudent"/>
    <ns1:name>UndergraduateStudent290</ns1:name>
    <ns1:memberOf rdf:resource="http://www.Department1.University0.edu"/>
    <ns1:emailAddress>UndergraduateStudent290@Department1.University0.edu</ns1:emailAddress>
    <ns1:telephone>xxx-xxx-xxxx</ns1:telephone>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course47"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course32"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/UndergraduateStudent37">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#UndergraduateStudent"/>
    <ns1:name>UndergraduateStudent37</ns1:name>
    <ns1:memberOf rdf:resource="http://www.Department1.University0.edu"/>
    <ns1:emailAddress>UndergraduateStudent37@Department1.University0.edu</ns1:emailAddress>
    <ns1:telephone>xxx-xxx-xxxx</ns1:telephone>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course22"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course42"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/AssociateProfessor9">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#AssociateProfessor"/>
    <ns1:name>AssociateProfessor9</ns1:name>
    <ns1:teacherOf rdf:resource="http://www.Department1.University0.edu/Course25"/>
    <ns1:teacherOf rdf:resource="http://www.Department1.University0.edu/GraduateCourse26"/>
    <ns1:undergraduateDegreeFrom rdf:resource="http://www.University681.edu"/>
    <ns1:mastersDegreeFrom rdf:resource="http://www.University614.edu"/>
    <ns1:doctoralDegreeFrom rdf:resource="http://www.University726.edu"/>
    <ns1:worksFor rdf:resource="http://www.Department1.University0.edu"/>
    <ns1:emailAddress>AssociateProfessor9@Department1.University0.edu</ns1:emailAddress>
    <ns1:telephone>xxx-xxx-xxxx</ns1:telephone>
    <ns1:researchInterest>Research27</ns1:researchInterest>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/UndergraduateStudent323">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#UndergraduateStudent"/>
    <ns1:name>UndergraduateStudent323</ns1:name>
    <ns1:memberOf rdf:resource="http://www.Department1.University0.edu"/>
    <ns1:emailAddress>UndergraduateStudent323@Department1.University0.edu</ns1:emailAddress>
    <ns1:telephone>xxx-xxx-xxxx</ns1:telephone>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course0"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course35"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course4"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course20"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/Lecturer2">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#Lecturer"/>
    <ns1:name>Lecturer2</ns1:name>
    <ns1:teacherOf rdf:resource="http://www.Department1.University0.edu/Course40"/>
    <ns1:teacherOf rdf:resource="http://www.Department1.University0.edu/Course41"/>
    <ns1:teacherOf rdf:resource="http://www.Department1.University0.edu/GraduateCourse40"/>
    <ns1:undergraduateDegreeFrom rdf:resource="http://www.University481.edu"/>
    <ns1:mastersDegreeFrom rdf:resource="http://www.University179.edu"/>
    <ns1:doctoralDegreeFrom rdf:resource="http://www.University154.edu"/>
    <ns1:worksFor rdf:resource="http://www.Department1.University0.edu"/>
    <ns1:emailAddress>Lecturer2@Department1.University0.edu</ns1:emailAddress>
    <ns1:telephone>xxx-xxx-xxxx</ns1:telephone>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/GraduateStudent93">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#GraduateStudent"/>
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#TeachingAssistant"/>
    <ns1:name>GraduateStudent93</ns1:name>
    <ns1:memberOf rdf:resource="http://www.Department1.University0.edu"/>
    <ns1:emailAddress>GraduateStudent93@Department1.University0.edu</ns1:emailAddress>
    <ns1:telephone>xxx-xxx-xxxx</ns1:telephone>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/GraduateCourse39"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/GraduateCourse17"/>
    <ns1:undergraduateDegreeFrom rdf:resource="http://www.University984.edu"/>
    <ns1:advisor rdf:resource="http://www.Department1.University0.edu/FullProfessor4"/>
    <ns1:teachingAssistantOf rdf:resource="http://www.Department1.University0.edu/Course10"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/UndergraduateStudent210">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#UndergraduateStudent"/>
    <ns1:name>UndergraduateStudent210</ns1:name>
    <ns1:memberOf rdf:resource="http://www.Department1.University0.edu"/>
    <ns1:emailAddress>UndergraduateStudent210@Department1.University0.edu</ns1:emailAddress>
    <ns1:telephone>xxx-xxx-xxxx</ns1:telephone>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course11"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course34"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course8"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course40"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/Lecturer3">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#Lecturer"/>
    <ns1:name>Lecturer3</ns1:name>
    <ns1:teacherOf rdf:resource="http://www.Department1.University0.edu/Course42"/>
    <ns1:teacherOf rdf:resource="http://www.Department1.University0.edu/Course43"/>
    <ns1:teacherOf rdf:resource="http://www.Department1.University0.edu/GraduateCourse41"/>
    <ns1:teacherOf rdf:resource="http://www.Department1.University0.edu/GraduateCourse42"/>
    <ns1:undergraduateDegreeFrom rdf:resource="http://www.University315.edu"/>
    <ns1:mastersDegreeFrom rdf:resource="http://www.University204.edu"/>
    <ns1:doctoralDegreeFrom rdf:resource="http://www.University34.edu"/>
    <ns1:worksFor rdf:resource="http://www.Department1.University0.edu"/>
    <ns1:emailAddress>Lecturer3@Department1.University0.edu</ns1:emailAddress>
    <ns1:telephone>xxx-xxx-xxxx</ns1:telephone>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/UndergraduateStudent84">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#UndergraduateStudent"/>
    <ns1:name>UndergraduateStudent84</ns1:name>
    <ns1:memberOf rdf:resource="http://www.Department1.University0.edu"/>
    <ns1:emailAddress>UndergraduateStudent84@Department1.University0.edu</ns1:emailAddress>
    <ns1:telephone>xxx-xxx-xxxx</ns1:telephone>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course21"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course23"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course47"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/UndergraduateStudent161">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#UndergraduateStudent"/>
    <ns1:name>UndergraduateStudent161</ns1:name>
    <ns1:memberOf rdf:resource="http://www.Department1.University0.edu"/>
    <ns1:emailAddress>UndergraduateStudent161@Department1.University0.edu</ns1:emailAddress>
    <ns1:telephone>xxx-xxx-xxxx</ns1:telephone>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course4"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course12"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/UndergraduateStudent344">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#UndergraduateStudent"/>
    <ns1:name>UndergraduateStudent344</ns1:name>
    <ns1:memberOf rdf:resource="http://www.Department1.University0.edu"/>
    <ns1:emailAddress>UndergraduateStudent344@Department1.University0.edu</ns1:emailAddress>
    <ns1:telephone>xxx-xxx-xxxx</ns1:telephone>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course11"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course35"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course31"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course44"/>
    <ns1:advisor rdf:resource="http://www.Department1.University0.edu/AssociateProfessor0"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/UndergraduateStudent109">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#UndergraduateStudent"/>
    <ns1:name>UndergraduateStudent109</ns1:name>
    <ns1:memberOf rdf:resource="http://www.Department1.University0.edu"/>
    <ns1:emailAddress>UndergraduateStudent109@Department1.University0.edu</ns1:emailAddress>
    <ns1:telephone>xxx-xxx-xxxx</ns1:telephone>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course38"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course27"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course30"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/AssociateProfessor0/Publication9">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#Publication"/>
    <ns1:name>Publication9</ns1:name>
    <ns1:publicationAuthor rdf:resource="http://www.Department1.University0.edu/AssociateProfessor0"/>
    <ns1:publicationAuthor rdf:resource="http://www.Department1.University0.edu/GraduateStudent54"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/UndergraduateStudent286">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#UndergraduateStudent"/>
    <ns1:name>UndergraduateStudent286</ns1:name>
    <ns1:memberOf rdf:resource="http://www.Department1.University0.edu"/>
    <ns1:emailAddress>UndergraduateStudent286@Department1.University0.edu</ns1:emailAddress>
    <ns1:telephone>xxx-xxx-xxxx</ns1:telephone>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course3"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course6"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course27"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course0"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.University666.edu">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#University"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/GraduateStudent60">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#GraduateStudent"/>
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#ResearchAssistant"/>
    <ns1:name>GraduateStudent60</ns1:name>
    <ns1:memberOf rdf:resource="http://www.Department1.University0.edu"/>
    <ns1:emailAddress>GraduateStudent60@Department1.University0.edu</ns1:emailAddress>
    <ns1:telephone>xxx-xxx-xxxx</ns1:telephone>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/GraduateCourse43"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/GraduateCourse20"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/GraduateCourse41"/>
    <ns1:undergraduateDegreeFrom rdf:resource="http://www.University605.edu"/>
    <ns1:advisor rdf:resource="http://www.Department1.University0.edu/AssociateProfessor2"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/GraduateStudent42">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#GraduateStudent"/>
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#ResearchAssistant"/>
    <ns1:name>GraduateStudent42</ns1:name>
    <ns1:memberOf rdf:resource="http://www.Department1.University0.edu"/>
    <ns1:emailAddress>GraduateStudent42@Department1.University0.edu</ns1:emailAddress>
    <ns1:telephone>xxx-xxx-xxxx</ns1:telephone>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/GraduateCourse4"/>
    <ns1:undergraduateDegreeFrom rdf:resource="http://www.University558.edu"/>
    <ns1:advisor rdf:resource="http://www.Department1.University0.edu/AssistantProfessor2"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/UndergraduateStudent195">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#UndergraduateStudent"/>
    <ns1:name>UndergraduateStudent195</ns1:name>
    <ns1:memberOf rdf:resource="http://www.Department1.University0.edu"/>
    <ns1:emailAddress>UndergraduateStudent195@Department1.University0.edu</ns1:emailAddress>
    <ns1:telephone>xxx-xxx-xxxx</ns1:telephone>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course44"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course34"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course38"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/Course8"/>
    <ns1:advisor rdf:resource="http://www.Department1.University0.edu/AssistantProfessor3"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/GraduateStudent85">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#GraduateStudent"/>
    <ns1:name>GraduateStudent85</ns1:name>
    <ns1:memberOf rdf:resource="http://www.Department1.University0.edu"/>
    <ns1:emailAddress>GraduateStudent85@Department1.University0.edu</ns1:emailAddress>
    <ns1:telephone>xxx-xxx-xxxx</ns1:telephone>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/GraduateCourse24"/>
    <ns1:undergraduateDegreeFrom rdf:resource="http://www.University566.edu"/>
    <ns1:advisor rdf:resource="http://www.Department1.University0.edu/FullProfessor3"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/GraduateStudent106">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#GraduateStudent"/>
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#TeachingAssistant"/>
    <ns1:name>GraduateStudent106</ns1:name>
    <ns1:memberOf rdf:resource="http://www.Department1.University0.edu"/>
    <ns1:emailAddress>GraduateStudent106@Department1.University0.edu</ns1:emailAddress>
    <ns1:telephone>xxx-xxx-xxxx</ns1:telephone>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/GraduateCourse10"/>
    <ns1:takesCourse rdf:resource="http://www.Department1.University0.edu/GraduateCourse36"/>
    <ns1:undergraduateDegreeFrom rdf:resource="http://www.University658.edu"/>
    <ns1:advisor rdf:resource="http://www.Department1.University0.edu/FullProfessor6"/>
    <ns1:teachingAssistantOf rdf:resource="http://www.Department1.University0.edu/Course34"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.University376.edu">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#University"/>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.Department1.University0.edu/FullProfessor4/Publication7">
    <rdf:type rdf:resource="file:///c:/Users/DELL/Desktop/Linh/CaoHoc/T6.GiaiThuat/Code/data/LUBM_Data/uba/univ-bench.owl#Publication"/>
    <ns1:name>Publication7</ns1:name>
    <ns1:publicationAuthor rdf:resource="http://www.Department1.University0.edu/FullProfessor4"/>
  </rdf:Description>
</rdf:RDF>"""

# Tạo thư mục và ghi dữ liệu vào file
os.makedirs("LUBM_Data/data_test", exist_ok=True)  # Tạo thư mục data_test nếu chưa tồn tại

# Đường dẫn của file RDF
file_path_test = os.path.join("LUBM_Data/data_test", "University0_1.rdf")

# Ghi dữ liệu RDF vào file
with open(file_path_test, "w", encoding="utf-8") as rdf_file:
    rdf_file.write(rdf_data_test)

print(f"File '{file_path_test}' đã được tạo thành công với dữ liệu RDF.")


File 'LUBM_Data/data_test\University0_1.rdf' đã được tạo thành công với dữ liệu RDF.
